In [1]:
from moviepy.editor import *
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Your database connection
engine = create_engine('mysql+pymysql://root@localhost:3306/music_development')
data_path = '../data/'

In [2]:
sql = '''
SELECT * FROM songs
WHERE name LIKE "imagine"
'''
songs = pd.read_sql(sql, engine)
songs

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,5,3,Imagine,4,185,Imagine.mp3,27,rAn-AWXtHv0,1971-10-01,5,0


In [3]:
songId = songs['id'].iloc[0]  # Get the first value
sql = f"""SELECT * FROM songs WHERE id = {songId}"""
songs = pd.read_sql(sql, engine)
print(songs[['id', 'name', 'youtube_code', 'released_date']])

   id     name youtube_code released_date
0   5  Imagine  rAn-AWXtHv0    1971-10-01


In [4]:
sql = f'SELECT * FROM lyrics WHERE song_id = {songId}'
lyrics = pd.read_sql(sql, engine)
print("Lyrics Content:")
print(lyrics.content.iloc[0])
print("\n" + "="*50 + "\n")

# NEW: Export lyrics to text file function
def export_lyrics_to_text(song_id, songs_df, lyrics_df, export_path='../data/lyrics_export/'):
    """
    Export lyrics from the existing DataFrames to text files
    """
    try:
        # Get song details
        song_row = songs_df[songs_df['id'] == song_id].iloc[0]
        lyric_row = lyrics_df[lyrics_df['song_id'] == song_id]
        
        if lyric_row.empty:
            print(f"No lyrics found for song ID: {song_id}")
            return None
        
        # Get artist details (you might need to query artists table)
        artist_query = f'SELECT * FROM artists WHERE id = {song_row["artist_id"]}'
        artist_df = pd.read_sql(artist_query, engine)
        
        if not artist_df.empty:
            artist_row = artist_df.iloc[0]
            artist_name = f"{artist_row.get('first_name', '')} {artist_row.get('last_name', '')}".strip()
            if not artist_name:
                artist_name = artist_row.get('band', 'Unknown_Artist')
        else:
            artist_name = 'Unknown_Artist'
        
        # Clean names for filename
        song_name = song_row['name'].replace('/', '_').replace('\\', '_')
        artist_name_clean = artist_name.replace('/', '_').replace('\\', '_')
        
        # Create export directory
        os.makedirs(export_path, exist_ok=True)
        
        # Create filename
        filename = f"{artist_name_clean} - {song_name}.txt"
        filepath = os.path.join(export_path, filename)
        
        # Write to text file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"Artist: {artist_name}\n")
            f.write(f"Song: {song_name}\n")
            f.write(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 50 + "\n\n")
            f.write(lyric_row['content'].iloc[0])
        
        print(f"✅ Successfully exported lyrics to: {filepath}")
        return filepath
        
    except Exception as e:
        print(f"❌ Error exporting lyrics: {e}")
        return None

# NEW: Batch export function for multiple songs
def export_all_lyrics(export_path='../data/lyrics_export/'):
    """
    Export lyrics for all songs that have lyrics in the database
    """
    try:
        # Query all songs with lyrics
        sql_all = """
        SELECT s.id, s.name, s.artist_id, l.content 
        FROM songs s 
        JOIN lyrics l ON s.id = l.song_id
        """
        all_lyrics_df = pd.read_sql(sql_all, engine  )
        
        if all_lyrics_df.empty:
            print("No songs with lyrics found in the database")
            return []
        
        # Create export directory with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        export_dir = os.path.join(export_path, f'batch_export_{timestamp}')
        os.makedirs(export_dir, exist_ok=True)
        
        exported_files = []
        
        for _, row in all_lyrics_df.iterrows():
            # Get artist details for each song
            artist_query = f'SELECT * FROM artists WHERE id = {row["artist_id"]}'
            artist_df = pd.read_sql(artist_query, engine  )
            
            if not artist_df.empty:
                artist_row = artist_df.iloc[0]
                artist_name = f"{artist_row.get('first_name', '')} {artist_row.get('last_name', '')}".strip()
                if not artist_name:
                    artist_name = artist_row.get('band', 'Unknown_Artist')
            else:
                artist_name = 'Unknown_Artist'
            
            # Clean names for filename
            song_name = row['name'].replace('/', '_').replace('\\', '_')
            artist_name_clean = artist_name.replace('/', '_').replace('\\', '_')
            
            # Create filename and filepath
            filename = f"{artist_name_clean} - {song_name}.txt"
            filepath = os.path.join(export_dir, filename)
            
            # Write to text file
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"Artist: {artist_name}\n")
                f.write(f"Song: {song_name}\n")
                f.write(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 50 + "\n\n")
                f.write(row['content'])
            
            exported_files.append(filepath)
            print(f"Exported: {filename}")
        
        print(f"\n✅ Successfully exported {len(exported_files)} lyrics files to: {export_dir}")
        return exported_files
        
    except Exception as e:
        print(f"❌ Error in batch export: {e}")
        return []

# Usage examples:

print(f"1. Export single song (ID={songId}):")
exported_file = export_lyrics_to_text(5, songs, lyrics)
if exported_file:
    print(f"Single file exported: {exported_file}")

print("\n" + "="*80 + "\n")

Lyrics Content:


Imagine there's no heaven
It's easy if you try
No hell below us
Above us only sky

Imagine all the people
Living for today

Imagine there's no countries
It isn't hard to do
Nothing to kill or die for
And no religion, too

Imagine all the people
Living life in peace

You may say I'm a dreamer
But I'm not the only one
I hope someday you will join us
And the world will be as one

Imagine no possessions
I wonder if you can
No need for greed or hunger
A brotherhood of man

Imagine all the people
Sharing all the world

You, you may say I'm a dreamer
But I'm not the only one
I hope someday you will join us
And the world will live as one




1. Export single song (ID=5):
✅ Successfully exported lyrics to: ../data/lyrics_export/John Lennon - Imagine.txt
Single file exported: ../data/lyrics_export/John Lennon - Imagine.txt




In [5]:
# Check the uploads directory structure
uploads_path = r"C:\ruby\music\public\uploads\song\location"

if os.path.exists(uploads_path):
    print("📁 Contents of uploads directory:")
    for item in os.listdir(uploads_path):
        item_path = os.path.join(uploads_path, item)
        if os.path.isdir(item_path):
            #print(f"📂 Folder: {item}")
            # Check inside folder songId
            if item == f"{songId}":
                for file in os.listdir(item_path):
                    print(f"   📄 File: {file}")
        else:
            print(f"📄 File: {item}")
else:
    print(f"❌ Directory not found: {uploads_path}")

📁 Contents of uploads directory:
   📄 File: AlbumArtSmall.jpg
   📄 File: AlbumArt_{95BDDABE-1ED8-437E-87BC-4823B7ED0A9B}_Large.jpg
   📄 File: AlbumArt_{95BDDABE-1ED8-437E-87BC-4823B7ED0A9B}_Small.jpg
   📄 File: desktop.ini
   📄 File: Folder-old.jpg
   📄 File: Folder.jpg
   📄 File: Imagine.mp3


In [6]:
def create_reliable_lyrics_video(song_id=songId, max_duration=60):
    """RELIABLE version - back to basics with proven working code"""
    
    try:
        # Get song data
        query = f"""
        SELECT s.name as song_name, s.location as audio_file,
               l.content as lyrics, a.first_name, a.last_name
        FROM songs s 
        JOIN lyrics l ON s.id = l.song_id 
        JOIN artists a ON s.artist_id = a.id 
        WHERE s.id = {song_id}
        """
        
        df = pd.read_sql(query, engine)
        song_data = df.iloc[0]
        
        print(f"🎵 Creating RELIABLE video for: {song_data['song_name']}")
        
        # Construct file paths
        audio_dir = os.path.join(r"C:\ruby\music\public\uploads\song\location", str(song_id))
        audio_path = os.path.join(audio_dir, song_data['audio_file'])
        background_image_path = os.path.join(audio_dir, "Folder.jpg")
        
        print(f"✅ Audio: {os.path.basename(audio_path)}")
        print(f"✅ Background: {os.path.basename(background_image_path)}")
        
        if not os.path.exists(audio_path):
            print("❌ Audio file not found")
            return None
        
        # Load audio
        audio_clip = AudioFileClip(audio_path)
    
        # Duration
        if max_duration:
            duration = min(max_duration, audio_clip.duration)
            print(f"⏱️ Using {duration:.1f}s (limited)")
        else:
            duration = audio_clip.duration
            print(f"⏱️ Using FULL duration: {duration:.1f}s")
    
        audio_clip = audio_clip.subclip(0, duration)
        
        print(f"⏱️ Song duration: {duration:.2f} seconds ({duration/60:.2f} minutes)")        
        
        # Video settings
        fps = 24
        width, height = 640, 480
        
        # **SIMPLE RELIABLE BACKGROUND LOADING**
        def make_frame(t):
            try:
                # Load background image fresh every time to avoid flipping
                if os.path.exists(background_image_path):
                    bg_image = Image.open(background_image_path)
                    bg_image = bg_image.resize((width, height), Image.Resampling.LANCZOS)
                    frame = np.array(bg_image)
                else:
                    # Fallback to solid color
                    frame = np.full((height, width, 3), [40, 40, 80], dtype=np.uint8)
                
                # Convert to PIL for text drawing
                pil_img = Image.fromarray(frame)
                draw = ImageDraw.Draw(pil_img)
                
                # Load font
                try:
                    font = ImageFont.truetype("arial.ttf", 32)
                except:
                    try:
                        font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", 32)
                    except:
                        font = ImageFont.load_default()
                
                # Parse lyrics
                lyrics_text = song_data['lyrics']
                lines = [line.strip() for line in lyrics_text.split('\n') if line.strip()]
                
                # Calculate current line
                if len(lines) > 0:
                    line_index = int((t / duration) * len(lines))
                    line_index = min(line_index, len(lines) - 1)
                    current_line = lines[line_index]
                    
                    # Calculate text position
                    try:
                        bbox = draw.textbbox((0, 0), current_line, font=font)
                    except AttributeError:
                        bbox = draw.textsize(current_line, font=font)
                        bbox = (0, 0, bbox[0], bbox[1])
                    
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    x = (width - text_width) // 2
                    y = (height - text_height) // 2
                    
                    # **IMPROVED TEXT READABILITY**
                    # Draw black background behind text
                    padding = 10
                    draw.rectangle([
                        x - padding, y - padding,
                        x + text_width + padding, y + text_height + padding
                    ], fill=(0, 0, 0, 128))  # Semi-transparent black
                    
                    # Draw text with shadow
                    shadow_color = (0, 0, 0)
                    text_color = (255, 255, 255)
                    
                    # Shadow
                    draw.text((x+2, y+2), current_line, font=font, fill=shadow_color)
                    # Main text
                    draw.text((x, y), current_line, font=font, fill=text_color)
                
                return np.array(pil_img)
                
            except Exception as e:
                print(f"❌ Frame error at {t:.1f}s: {e}")
                # Return black frame as fallback
                return np.zeros((height, width, 3), dtype=np.uint8)
        
        # **CREATE VIDEO - SIMPLE AND RELIABLE**
        print("🎬 Creating reliable video frames...")
        video = VideoClip(make_frame, duration=duration)
        
        # Add audio
        video = video.set_audio(audio_clip)
        
        # Export
        output_dir = '../data/videos'
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{song_data['song_name']}_reliable.mp4")
        
        print("📹 Exporting reliable video...")
        video.write_videofile(
            output_file, 
            fps=fps, 
            codec='libx264',
            audio_codec='aac',
            verbose=False,
            logger=None
        )
        
        print(f"✅ RELIABLE video created: {output_file}")
        print(f"📊 File size: {os.path.getsize(output_file) / (1024*1024):.1f} MB")
        
        # Verify the video
        try:
            verify_clip = VideoFileClip(output_file)
            has_audio = verify_clip.audio is not None
            verify_duration = verify_clip.duration
            verify_clip.close()
            
            print(f"🔍 Verification - Audio: {'✅ YES' if has_audio else '❌ NO'}, Duration: {verify_duration:.1f}s")
        except:
            print("⚠️ Could not verify final file")
        
        # Clean up
        video.close()
        audio_clip.close()
        
        return output_file
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

In [7]:
# Run the reliable version
print("=" * 60)
print("🎬 CREATING RELIABLE VIDEO (BACK TO BASICS)")
print("=" * 60)
result = create_reliable_lyrics_video(song_id=songId, max_duration=45)

🎬 CREATING RELIABLE VIDEO (BACK TO BASICS)
🎵 Creating RELIABLE video for: Imagine
✅ Audio: Imagine.mp3
✅ Background: Folder.jpg
⏱️ Using 45.0s (limited)
⏱️ Song duration: 45.00 seconds (0.75 minutes)
🎬 Creating reliable video frames...
📹 Exporting reliable video...
✅ RELIABLE video created: ../data/videos\Imagine_reliable.mp4
📊 File size: 1.1 MB
🔍 Verification - Audio: ✅ YES, Duration: 45.0s
